In [106]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [107]:
from lib.Stream import Stream
from lib.TimeNode import TimeNode, TimeNodeSet
from lib.StreamProperties import StreamStarSat
from lib.Graph import Graph, StarSat, BHACore
from lib.patterns import *
import networkx as nx
import matplotlib.pyplot as plt
import logging
import StreamFig
from IPython.display import Image, clear_output
print(StreamFig.__version__)

1.1d


Faire des minidonnées de test, calculer l'intérieur (pour des propriétés de degrés genre hub-autorité etc.)

In [108]:
basename = "ChangingNeighbours-StSa-Copy1"
# basename = "testPathos1"

s = Stream(lang=set("abcd"), _loglevel=logging.DEBUG)
core_property = StreamStarSat(s, threshold=2)
s.setCoreProperty(core_property)
# s.readStream("./tests/integration/fixtures/ChangingNeighbours-StSa-Copy1.json")
s.readStream(f"./tests/integration/fixtures/{basename}.json")

s.T

{'alpha': 0, 'omega': 10}

In [109]:
I = set()
for l in s.E:
    I = I.union(l["label_u"])
    I = I.union(l["label_v"])
s.I = I
s.I

{'a', 'b', 'c', 'd'}

In [110]:
X1 = [TimeNode(x["u"], x["b"], x["e"]) for x in s.E ]
X2 = [TimeNode(x["v"], x["b"], x["e"]) for x in s.E ]

X = X1 + X2
X = TimeNodeSet(elements=X)
s.W = X
#interior = interior(s, X, X)
# print(interior)

In [111]:
res_int = interior(s)
res_int

T: {'alpha': 0, 'omega': 10}
V: {'v', 'u', 'x', 'y'},
W: {u [1, 4], v [1, 4], x [1, 3], y [2, 4]},
E: (1, 4, u, v)
(1, 3, v, x)
(2, 4, v, y)        

In [112]:
def mydraw(s, q, node_clusters=[]):
    basename = "ChangingNeighbours-StSa-Copy1"
    pattern_str = ''.join(sorted(q))
    
    s_draw = StreamFig.StreamFig(alpha=0, omega=5, streaming=False)
    _ = [s_draw.addNode(x) for x in s.nodes()]
    for l in s.E:
        if set([l["u"], l["v"]]) == set(["v", "y"]):
            s_draw.addLink(l["u"], l["v"], l["b"], l["e"], curving=0.2)
        else:
            s_draw.addLink(l["u"], l["v"], l["b"], l["e"])

    for nc in node_clusters:
        v, b, e, color = nc
        s_draw.addNodeCluster(v, [(b, e)], color=color)
    # s_draw.addTimeLine(ticks=2)
    s_draw.save(f"{basename}-{pattern_str}.fig")
    # For display
    from subprocess import call
    call(f"rm {basename}-{pattern_str}.png", shell=True)
    call(f"fig2dev -Lpng {basename}-{pattern_str}.fig > {basename}-{pattern_str}.png", shell=True)
    call(f"rm {basename}-{pattern_str}.fig", shell=True)
    
    return f"{basename}-{pattern_str}.png"


#my_nc = [ (x.node, x.b, x.e, 11) for x in res_int[0] ] 
#my_nc += [ (x.node, x.b, x.e, 12) for x in res_int[1] ] 
#mydraw(s, set('ab'), node_clusters=my_nc)

In [113]:
bipatterns(s, X, X)

{'b', 'a'} {u [1, 4], v [1, 4], x [1, 3], y [2, 4]}
{'b', 'c', 'a'} {u [1, 3], v [1, 3], x [1, 3]}
{'b', 'a', 'd'} {u [2, 4], v [2, 4], y [2, 4]}


In [114]:
s.bipatterns_list

[({'b', 'a'} {u [1, 4], v [1, 4], x [1, 3], y [2, 4]}, set()),
 ({'b', 'c', 'a'} {u [1, 3], v [1, 3], x [1, 3]}, {'a', 'b'}),
 ({'b', 'a', 'd'} {u [2, 4], v [2, 4], y [2, 4]}, {'a', 'b'})]

In [35]:
outf = open("bipatterns.dot", "w+")

print("digraph bips {", file=outf)

nodes = [ bp for bp in s.bipatterns_list]
edges = []

for x in s.bipatterns_list:
    bp, bp_parent = x
    basename = "ChangingNeighbours-StSa-Copy1"
    pattern_str = ''.join(sorted(bp.lang))
    
    subs = s.substream(bp.support_set[0], bp.support_set[1])
    
    my_nc = [ (x.node, x.b, x.e, 11) for x in bp.support_set[0] ] 
    my_nc += [ (x.node, x.b, x.e, 12) for x in bp.support_set[1] ]
    fname = mydraw(subs, bp.lang, node_clusters=my_nc)
    print(f'{pattern_str}[shape="plaintext", image="{basename}-{pattern_str}.png", label="{pattern_str}", labelloc="b", imagepos="tc"];', file=outf)
    
    edges.append((pattern_str, ''.join(sorted(bp_parent))))

for e in edges:
    if not (e[0] == '' or e[1] == ''):
        print(f'{e[1]} -> {e[0]}', file=outf)
print("}", file=outf)
outf.close()
from subprocess import call
call(f"dot -Tps bipatterns.dot -o outfile.ps", shell=True)

1

In [36]:
s = Stream(lang=set("abcd"), _loglevel=logging.DEBUG)
core_property = StreamStarSat(s, threshold=2)
s.setCoreProperty(core_property)
s.readStream("./tests/integration/fixtures/testJaccard.json")

def jaccard(s, u, v):
    """ 
        u,v : two nodes
        
        @return: the Jaccard coefficient of u and v
    """
    jaccard = 0.0
    
    n_u = TimeNodeSet(elements=[ TimeNode(x, b, e) for x in s.neighbours(u) for b,e,l_u,l_v in s.times[frozenset([u, x])] ])
    n_v = TimeNodeSet(elements=[ TimeNode(x, b, e) for x in s.neighbours(v) for b,e,l_u,l_v in s.times[frozenset([v, x])] ])
    
    # Union
    union = sum(( x.e - x.b for x in n_u.union(n_v) ))
    
    # Intersection
    inter = sum( ( x.e - x.b for x in n_u.intersection(n_v) ) )
    
    return inter / union

jaccard(s, "v", "x")

0.25